In [2]:
import numpy as np

import io
import time

import motor

PROGRAM_START_NS = time.time_ns()

In [2]:
mm = motor.MotorManager()
my_motor = mm.get_connected_motors()[-1]

In [14]:
mm.commands()

[<Command: 0>, <Command: 0>]

In [16]:
api_options = my_motor.get_api_options()

[<Status at: 1942595902>]

In [3]:
def print_motor_status(motor_manager):
    statuses = motor_manager.read()
    for m, status in enumerate(statuses):
        error_bits = status.flags.error.bits
        errors = [k for k, v in error_bits.items() if v]
        if error_bits['fault']:
            print(errors)
        else:
            print('operational')

In [4]:
def now_ms():
    return int((time.time_ns() - PROGRAM_START_NS) / 1000)

In [5]:
print_motor_status(mm)

operational


In [73]:
mm.write([motor.Command(host_timestamp=now_ms(), mode=motor.DriverDisable)])

In [75]:
mm.write([motor.Command(host_timestamp=now_ms(), mode=motor.DriverEnable)])

In [79]:
mm.write([motor.Command(host_timestamp=now_ms(), mode=motor.ClearFaults)])

In [100]:
mm.write([motor.Command(host_timestamp=now_ms(), mode=motor.Open)])

In [81]:
mm.write([motor.Command(host_timestamp=now_ms(), mode=motor.Damped)])

In [85]:
c = motor.Command(host_timestamp=now_ms(), mode=motor.PhaseLock)
c.current_desired = 0.4
mm.write([c])

In [96]:
c = motor.Command(host_timestamp=now_ms(), mode=motor.Current)
c.current_desired = -0.1
mm.write([c])

In [10]:
c = motor.Command(host_timestamp=now_ms(), mode=motor.CurrentTuning)
c.current_tuning.amplitude = 0.1
c.current_tuning.frequency = 2000
c.current_tuning.mode = motor.Square
mm.write([c])

In [99]:
flogs = []
for freq in np.linspace(100, 2000, 50):
    c = motor.Command(host_timestamp=now_ms(), mode=motor.CurrentTuning)
    c.current_tuning.amplitude = 0.1
    c.current_tuning.frequency = freq
    c.current_tuning.mode = motor.Square
    mm.write([c])
    for _ in range(2):
        flogs.append(my_motor.get_fast_log())
mm.write([motor.Command(host_timestamp=now_ms(), mode=motor.Open)])

In [7]:
c = motor.Command(host_timestamp=now_ms(), mode=motor.CurrentTuning)
c.current_tuning.amplitude = 0.1
c.current_tuning.frequency = 1000
c.current_tuning.mode = motor.Square
mm.write([c])
time.sleep(0.5)

flogs = []
for _ in range(10):
    flogs.append(my_motor.get_fast_log())
mm.write([motor.Command(host_timestamp=now_ms(), mode=motor.Open)])

In [8]:
flogs

[{'ok': []},
 {'ok': []},
 {'ok': []},
 {'ok': []},
 {'ok': []},
 {'ok': []},
 {'ok': []},
 {'ok': []},
 {'ok': []},
 {'ok': []}]

In [8]:
from bokeh.io import output_notebook
from bokeh.models import BoxZoomTool, CrosshairTool, HoverTool, PanTool, ResetTool
from bokeh.plotting import figure, show

output_notebook()

Loading BokehJS ...

In [107]:
# create a plot
hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("(x,y)", "(@x, @y)"),
])

p = figure(
    tools=[BoxZoomTool(), CrosshairTool(), hover, PanTool(dimensions='width'), ResetTool()],
    width=800,
    height=400,
)

# add a renderer
for flog in flogs:
    y = np.array(flog['iq_meas_filt'])
    y2 = np.array(flog['iq_des'])
    threshold = 0.5 * np.max(y2)
    rising_edge_index = np.argmax(y2 > threshold)
    falling_edge_index = np.argmax(-y2[rising_edge_index:] > threshold)
    x = np.array(flog['timestamp'])
    x -= x[rising_edge_index:][falling_edge_index]
    
    p.step(x, y, mode='after')
    p.circle(x, y, size=2)
    p.step(x, y2, mode='after', color='black', line_dash='dotted')


# show the results
show(p)

In [17]:
import dataclasses

@dataclasses.dataclass
class BodePoint:
    frequency: float
    gain: float
    phase: float

def compute_magnitude_phase(signal: np.ndarray, sample_period: float) -> BodePoint:
    num_samples = len(signal)
    num_halfwidth = int(0.5 * num_samples)
    fft_freq = np.fft.fftfreq(num_samples, sample_period)

    dft = np.fft.fft(signal)
    idx_max_dft = np.argmax(np.abs(dft[1:num_halfwidth])) + 1
    max_fft = dft[idx_max_dft]
    gain = np.abs(max_fft) / num_halfwidth
    phase = np.angle(max_fft)
    frequency = fft_freq[idx_max_dft]

    
    return BodePoint(frequency, gain, phase)

class MotorOperator:
    def __init__(self):
        self.motor_manager = motor.MotorManager()
        self.start_time = time.time_ns()
        self.times = []

    def _pseudo_time(self):
        self.times.append(time.time_ns())
        return len(self.times)

    def _send_command(self, command):
        return self.motor_manager.write([command])
    
    @property
    def current_motor(self):
        return self.motor_manager.motors()[0]
    
    def get_fast_log(self):
        return self.current_motor.get_fast_log()

    def brake(self):
        return self._send_command(
            motor.Command(host_timestamp=self._pseudo_time(), mode=motor.Damped)
        )

    def open(self):
        return self._send_command(
            motor.Command(host_timestamp=self._pseudo_time(), mode=motor.Open)
        )

    def current(self, magnitude):
        command = motor.Command(
            host_timestamp=self._pseudo_time(), mode=motor.Current
        )
        command.current_desired = magnitude
        return self._send_command(command)

    def phase_lock(self, magnitude):
        command = motor.Command(
            host_timestamp=self._pseudo_time(), mode=motor.PhaseLock
        )
        command.current_desired = magnitude
        return self._send_command(command)

    def current_tuning(self, amplitude, frequency, mode=motor.Square, bias=0.0):
        command = motor.Command(
            host_timestamp=self._pseudo_time(), mode=motor.CurrentTuning
        )
        command.current_tuning.amplitude = amplitude
        command.current_tuning.frequency = frequency
        command.current_tuning.bias = bias
        command.current_tuning.mode = mode
        return self._send_command(command)
    
    def current_tuning_bode(self, amplitude, bias=0.0, start_frequency_hz=100, end_frequency_hz=10000, increment_hz=100) -> list[BodePoint]:
        bode_points = []
        cpu_frequency = self.current_motor.get_cpu_frequency()
        for f in np.arange(start_frequency_hz, end_frequency_hz, increment_hz):
            self.current_tuning(amplitude=amplitude, frequency=f, mode=motor.Sine, bias=bias)
            time.sleep(0.5)
            fast_log = self.get_fast_log()
            time_s = np.array(fast_log["timestamp"]) / cpu_frequency
            iq_des = np.array(fast_log["iq_des"])
            iq_mes = np.array(fast_log["iq_meas_filt"])
            sample_period = np.mean(np.diff(time_s))
            bode_iq_des = compute_magnitude_phase(iq_des, sample_period)
            bode_iq_mes = compute_magnitude_phase(iq_mes, sample_period)
            print(f'{f} Hz - m={bode_iq_mes.frequency} d={bode_iq_des.frequency}')
            bode_points.append(BodePoint(f, bode_iq_mes.gain / bode_iq_des.gain, bode_iq_mes.phase - bode_iq_des.phase))
        return bode_points





In [20]:
mo = MotorOperator()
bps = mo.current_tuning_bode(0.2)
mo.open()

100 Hz - m=526.31743627405 d=526.31743627405
200 Hz - m=526.31743627405 d=526.31743627405
300 Hz - m=526.3157894737174 d=526.3157894737174
400 Hz - m=526.3157894737174 d=526.3157894737174
500 Hz - m=526.3339048461819 d=526.3339048461819
600 Hz - m=526.3207299066264 d=526.3207299066264
700 Hz - m=526.3157894737174 d=526.3157894737174
800 Hz - m=526.3157894737174 d=526.3157894737174
900 Hz - m=1052.6315789474347 d=1052.6315789474347
1000 Hz - m=1052.6315789474347 d=1052.6315789474347
1100 Hz - m=1052.6315789474347 d=1052.6315789474347
1200 Hz - m=1052.6315789474347 d=1052.6315789474347
1300 Hz - m=1052.6315789474347 d=1052.6315789474347
1400 Hz - m=1578.9473684210586 d=1578.9473684210586
1500 Hz - m=1578.9523088225228 d=1578.9523088225228
1600 Hz - m=1578.9523088225228 d=1578.9523088225228
1700 Hz - m=1578.9572492551831 d=1578.9572492551831
1800 Hz - m=1578.9523088225228 d=1578.9523088225228
1900 Hz - m=2105.2631578948694 d=2105.2631578948694
2000 Hz - m=2105.276332339747 d=2105.27633233

In [23]:
#  create a plot
hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("(x,y)", "(@x, @y)"),
])

p = figure(
    tools=[BoxZoomTool(), CrosshairTool(), hover, PanTool(dimensions='width'), ResetTool()],
    width=800,
    height=400,
    x_axis_type='log'
)

# add a renderer
x = [p.frequency for p in bps]
y = 20 * np.log10([p.gain for p in bps])
p.line(x,y)



# show the results
show(p)

In [24]:
10**(-3 / 20)

0.7079457843841379

In [19]:
#  create a plot
hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("(x,y)", "(@x, @y)"),
])

p = figure(
    tools=[BoxZoomTool(), CrosshairTool(), hover, PanTool(dimensions='width'), ResetTool()],
    width=800,
    height=400,
    x_axis_type='log'
)

# add a renderer
x = [p.frequency for p in bps]
y = 20 * np.log10([p.gain for p in bps])
p.step(x,y, mode='after')



# show the results
show(p)

In [16]:
#  create a plot
hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("(x,y)", "(@x, @y)"),
])

p = figure(
    tools=[BoxZoomTool(), CrosshairTool(), hover, PanTool(dimensions='width'), ResetTool()],
    width=800,
    height=400,
    x_axis_type='log'
)

# add a renderer
x = [p.frequency for p in bps]
y = np.rad2deg(np.unwrap([p.phase for p in bps]))
p.step(x,y, mode='after')



# show the results
show(p)

In [8]:
class NoteComposer:
    def __init__(self):
        self.base_a = 220
        self.note_map = {'A':0,
                         'AS':1, 'BF':1,
                         'B':2, 'CF':2,
                         'BS':3, 'C':3,
                         'CS':4, 'DF':4, 'D':5,
                         'DS':6, 'EF':6,
                         'E':7, 'FF':7,
                         'ES':8, 'F':8,
                         'FS':9, 'GF':9, 'G':10,
                         'GS':11, 'AF':11}
        self.measure_length_s = 1.7
        self.spin_resolution = 0.01 * self.measure_length_s
        self.length_map = {'w':1.0, 'h':0.5, 'qes':0.4375, 'qe':0.375, 'qs':0.3125, 'q':0.25, 'es':0.1875, 'e':0.125, 's':0.0625, 't':0.03125, 'tr':0.25/3}
        self.current_amplitude_a = 0.1
        self.motor_manager = motor.MotorManager()
        self.start_time = time.time_ns()

    def _now_ms(self):
        return int((time.time_ns() - self.start_time) / 1000)
        

    def rest(self):
        return self._send_command(motor.Command(host_timestamp=self._now_ms(), mode=motor.Open))
    
    def tone(self, frequency, bias=0.0):
        command = motor.Command(host_timestamp=self._now_ms(), mode=motor.CurrentTuning)
        command.current_tuning.amplitude = self.current_amplitude_a 
        command.current_tuning.frequency = frequency
        command.current_tuning.mode = motor.Square
        command.current_tuning.bias = bias
        return self._send_command(command)

    def _send_command(self, command):
        return self.motor_manager.write([command])

    def pitch(self, p):
        return self.base_a * np.power(2, p / 12.0)
    
    def note(self, name, length='q', octave=0, bias=0.0):
        end_time = time.time() + self.length_map[length] * self.measure_length_s
        # Sing!
        if name in ['r', 'rest']:
            self.rest()
        else:
            self.tone(self.pitch(12 * octave + self.note_map[name.upper()]), bias)
        while time.time() < end_time - self.spin_resolution:
            time.sleep(self.spin_resolution)
        # Take  breath.
        self.rest()
        while time.time() < end_time:
            time.sleep(self.spin_resolution)

    def play(self, song, bias=0.0):
        note_list = [n.rstrip().lstrip() for n in song.split('\n') if n]
        for note in note_list:
            name, length, octave = note.split(':')
            self.note(name, length, int(octave), bias)


star_wars_theme ="""
r:h:0
r:q:0
d:tr:0
d:tr:0
d:tr:0

g:h:0
d:h:1

c:tr:1
b:tr:1
a:tr:1
g:h:1
d:q:1

c:tr:1
b:tr:1
a:tr:1
g:h:1
d:q:1

c:tr:1
b:tr:1
c:tr:1
a:h:1
d:tr:0
d:tr:0
d:tr:0

g:h:0
d:h:1

c:tr:1
b:tr:1
a:tr:1
g:h:1
d:q:1

c:tr:1
b:tr:1
a:tr:1
g:h:1
d:q:1

c:tr:1
b:tr:1
c:tr:1
a:h:1
d:qes:0
d:s:0

e:qe:0
e:e:0
c:e:1
b:e:1
a:e:1
g:e:0

g:tr:0
a:tr:1
b:tr:1
a:es:1
e:s:0
fs:q:0
d:qes:0
d:s:0

e:qe:0
e:e:0
c:e:1
b:e:1
a:e:1
g:e:0

d:qes:1
a:s:1
a:h:1
r:e:0
d:e:0

e:qe:0
e:e:0
c:e:1
b:e:1
a:e:1
g:e:0

g:tr:0
a:tr:1
b:tr:1
a:es:1
e:s:0
fs:q:0
d:qes:1
d:s:1

g:q:1
f:e:1
ef:q:1
d:e:1
c:q:1
bf:e:1
a:es:1
g:s:0

d:w:1

"""



In [9]:
nc = NoteComposer()
nc.play(star_wars_theme)
# nc.play(star_wars_theme, bias=-0.1)



In [15]:
T = 1.0
Fs = 100
f = 10
t = np.arange(0, T, 1 / Fs)
s = np.sin(2*np.pi*f*t)
dft = np.fft.fft(s)
n = len(dft)
hn = int(0.5*n - 1)
ff = np.linspace(0, 0.5*Fs, hn)
mdfth = np.abs(dft[:hn])

In [25]:
M = np.exp(2j*np.pi*np.outer(np.arange(len(s)), np.arange(len(s))) / len(s))

dft2 = M @ s
mdft2h = np.abs(dft2[:hn])

In [30]:
len(dft2)

100

In [31]:
np.fft.fftfreq(n)[:11] * Fs

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [26]:
print(ff[np.argmax(mdfth)])
print(ff[np.argmax(mdft2h)])

10.416666666666668
10.416666666666668
